# Imports

In [546]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_io as tfio

import matplotlib.pyplot as plt
import librosa.display
import IPython.display as ipd

import librosa

from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout
from keras.models import Sequential

from pathlib import Path


# Constants

In [547]:
DURATION = 20
OFFSET = 10
SAMPLING_RATE = 16000


In [548]:
p = "audio/classical/5.mp3"

In [549]:
# def load_audio(file_path):
#     data = tf.io.read_file(file_path)
#     wave, _ = librosa.stft(data)
#     return wave

In [550]:
def load_audio(
    file_path, sampling_rate=SAMPLING_RATE, duration=DURATION, offset=OFFSET
):
    if duration + offset >= int(librosa.get_duration(filename=file_path)):
        offset = 0
    data, _ = librosa.load(
        path=str(file_path), sr=sampling_rate, duration=duration, offset=offset
    )
    return data

In [551]:
# data = load_audio(p)
# ipd.Audio(data, rate=SAMPLING_RATE)

In [552]:
def get_spec(y):
    return np.abs(librosa.stft(y, n_fft=255, hop_length=512))

In [553]:
# d = get_spec(data)
# d.shape


In [554]:
def load_csv(path):
    data_csv = pd.read_csv(path, skipinitialspace=True)
    labels = data_csv.drop(
        columns=["mood", "age", "gender", "disliked", "liked", "mother tongue", "genre"]
    )  # unnecessary columns
    labels = labels.groupby("track id").mean()
    for column in labels.columns:
        labels[column] = labels[column].apply(lambda x: 1 if (x >= 0.5) else 0)

    return labels


y = load_csv("data.csv")
target = tf.data.Dataset.from_tensor_slices(y)

In [555]:
def dataset_of_audio_files():
    file_paths = []
    audio_path = Path("audio")
    genres = audio_path.glob("*")
    for genre in genres:
        for i in range(1, 101, 1):
            path = Path(f"{genre}/{i}.mp3")
            file_paths.append(str(path))
    #file_paths = tf.data.Dataset.list_files(file_paths, shuffle=False)
    return file_paths


files = dataset_of_audio_files()
for i in files:
    print(i)

audio\classical\1.mp3
audio\classical\2.mp3
audio\classical\3.mp3
audio\classical\4.mp3
audio\classical\5.mp3
audio\classical\6.mp3
audio\classical\7.mp3
audio\classical\8.mp3
audio\classical\9.mp3
audio\classical\10.mp3
audio\classical\11.mp3
audio\classical\12.mp3
audio\classical\13.mp3
audio\classical\14.mp3
audio\classical\15.mp3
audio\classical\16.mp3
audio\classical\17.mp3
audio\classical\18.mp3
audio\classical\19.mp3
audio\classical\20.mp3
audio\classical\21.mp3
audio\classical\22.mp3
audio\classical\23.mp3
audio\classical\24.mp3
audio\classical\25.mp3
audio\classical\26.mp3
audio\classical\27.mp3
audio\classical\28.mp3
audio\classical\29.mp3
audio\classical\30.mp3
audio\classical\31.mp3
audio\classical\32.mp3
audio\classical\33.mp3
audio\classical\34.mp3
audio\classical\35.mp3
audio\classical\36.mp3
audio\classical\37.mp3
audio\classical\38.mp3
audio\classical\39.mp3
audio\classical\40.mp3
audio\classical\41.mp3
audio\classical\42.mp3
audio\classical\43.mp3
audio\classical\44.m

In [556]:
data = []
for file in files:
    data.append(load_audio(file))
    #print(data)


In [557]:
files = tf.data.Dataset.from_tensor_slices(data)

In [558]:
df = tf.data.Dataset.zip((files, target))


In [564]:
def preprocess(data, label):
    # print(path)
    # print(label)
    #data = load_audio(path)
    print(type(data))
    spec = get_spec(data)
    spec = tf.expand_dims(spec, axis=2)
    return spec, label
    

In [565]:
filepath, label = df.shuffle(buffer_size=10000).as_numpy_iterator().next()

In [566]:
spectrogram, label = preprocess(filepath, label)
spectrogram, label

<class 'numpy.ndarray'>


(<tf.Tensor: shape=(128, 625, 1), dtype=float32, numpy=
 array([[[3.9537158e+00],
         [1.4197346e+00],
         [3.7999487e+00],
         ...,
         [1.0299018e-02],
         [4.7554126e+00],
         [1.0526814e+00]],
 
        [[4.8520637e+00],
         [1.1690910e+01],
         [7.0834265e+00],
         ...,
         [6.2473249e+00],
         [5.2611828e+00],
         [7.9552037e-01]],
 
        [[4.7215958e+00],
         [2.2125275e+01],
         [6.5025191e+00],
         ...,
         [5.3360944e+00],
         [5.2583590e+00],
         [3.2273858e+00]],
 
        ...,
 
        [[4.9004592e-03],
         [4.5990077e-05],
         [2.8599301e-05],
         ...,
         [3.1128709e-04],
         [5.4183812e-04],
         [7.0899630e-05]],
 
        [[3.0781729e-03],
         [1.4840771e-05],
         [3.4470074e-06],
         ...,
         [1.2459928e-04],
         [9.3565359e-05],
         [1.8952254e-05]],
 
        [[1.7276552e-03],
         [6.7876645e-06],
         [3.

In [567]:
df

<ZipDataset element_spec=(TensorSpec(shape=(320000,), dtype=tf.float32, name=None), TensorSpec(shape=(9,), dtype=tf.int64, name=None))>

In [568]:
data = df.map(preprocess)

<class 'tensorflow.python.framework.ops.Tensor'>


ParameterError: in user code:

    File "C:\Users\barto\AppData\Local\Temp\ipykernel_26276\1592215474.py", line 6, in preprocess  *
        spec = get_spec(data)
    File "C:\Users\barto\AppData\Local\Temp\ipykernel_26276\1248129145.py", line 2, in get_spec  *
        return np.abs(librosa.stft(y, n_fft=255, hop_length=512))
    File "e:\coding\github\audio-music-emotion-classification\venv\lib\site-packages\librosa\util\decorators.py", line 662, in inner_f  *
        return f(*args, **kwargs)
    File "e:\coding\github\audio-music-emotion-classification\venv\lib\site-packages\librosa\core\spectrum.py", line 202, in stft  *
        util.valid_audio(y, mono=False)
    File "e:\coding\github\audio-music-emotion-classification\venv\lib\site-packages\librosa\util\decorators.py", line 662, in inner_f  *
        return f(*args, **kwargs)
    File "e:\coding\github\audio-music-emotion-classification\venv\lib\site-packages\librosa\util\utils.py", line 272, in valid_audio  *
        raise Param

In [ ]:
for i in data:
    print(i)

(<tf.Tensor: shape=(2647296, 2), dtype=float32, numpy=
array([[ 0.0000000e+00,  0.0000000e+00],
       [ 6.9058867e-15,  8.9184777e-15],
       [ 8.9458385e-15,  6.1997518e-16],
       ...,
       [-3.4307002e-04, -1.2064925e-03],
       [-1.9574368e-03, -2.5921764e-03],
       [-2.7026713e-03, -2.9851031e-03]], dtype=float32)>, <tf.Tensor: shape=(9,), dtype=int64, numpy=array([0, 0, 0, 0, 1, 0, 0, 0, 0], dtype=int64)>)
(<tf.Tensor: shape=(2647296, 2), dtype=float32, numpy=
array([[ 0.00000000e+00,  0.00000000e+00],
       [-1.00273895e-14, -2.51377237e-14],
       [-8.34523626e-15, -1.32214084e-14],
       ...,
       [ 1.24788529e-03,  5.94543410e-04],
       [ 9.44934145e-04,  5.10683865e-04],
       [ 2.78530526e-04,  1.59415789e-03]], dtype=float32)>, <tf.Tensor: shape=(9,), dtype=int64, numpy=array([0, 0, 0, 0, 1, 0, 0, 0, 0], dtype=int64)>)
(<tf.Tensor: shape=(2647296, 2), dtype=float32, numpy=
array([[ 0.0000000e+00,  0.0000000e+00],
       [-2.0736669e-14, -1.9630368e-14],
   

KeyboardInterrupt: 